In [7]:
import argparse
import pandas as pd
import numpy as np
import os

os.getcwd()

'/Users/rdubey/mysrc/random-stuff/nlp-transformers'

In [8]:
!ls ./hypothesis/

task_1_ranking_model_baseline_triplet_cosine_us.csv
task_1_ranking_model_baseline_triplet_product_cosine_us.csv
task_1_ranking_model_baseline_triplet_us.csv
task_1_ranking_model_baseline_us.csv
task_1_ranking_model_fine_tuned_triplet_us.csv
task_1_ranking_model_fine_tuned_triplet_v1us.csv
task_1_ranking_model_fine_tuned_triplet_v3us.csv


In [10]:
dataset_path = '../../random-stuff/esci-data/shopping_queries_dataset/'
hypothesis_folder_path = './hypothesis'
output_path = './output'
""" 0. Init variables """

col_iteration = "iteration"
col_query_id = "query_id"
col_product_id = "product_id" 
col_product_locale = "product_locale"
col_small_version = "small_version"
col_split = "split"
col_esci_label = "esci_label"
col_relevance_pos = "relevance_pos"
col_ranking_postion = "ranking_postion"
col_score = "score"
col_conf = "conf"

max_trec_eval_score = 128
min_trec_eval_score = 0

esci_label2relevance_pos = {
    "E" : 4,
    "S" : 2,
    "C" : 3,
    "I" : 1,
}

""" 1. Generate RESULTS file """

locales = [
    "us",
#     "es",
#     "jp",
]

df_results = pd.DataFrame()
for locale in locales:
    df_ = pd.read_csv(
        os.path.join(hypothesis_folder_path, f"task_1_ranking_model_fine_tuned_triplet_v3_{locale}.csv"),
    )
    df_results = pd.concat([df_results, df_])

df_results_product_id = df_results.groupby(by=[col_query_id])
l_query_id = []
l_product_id = []
l_ranking_postion = []
l_score = []
for (query_id, rows) in df_results_product_id:
    n = len(rows)
    l_query_id += [query_id for _ in range(n)]
    l_product_id += rows[col_product_id].to_list()
    l_ranking_postion += [i for i in range(n)]
    l_score += list(np.arange(min_trec_eval_score, max_trec_eval_score, max_trec_eval_score / n).round(3)[::-1][:n])

df_res = pd.DataFrame({
    col_query_id : l_query_id,
    col_product_id : l_product_id,
    col_ranking_postion : l_ranking_postion,
    col_score : l_score,
})
model_name_value = "baseline"
iteration_value = "Q0"
df_res[col_conf] = model_name_value
df_res[col_iteration] = iteration_value

df_res[[
    col_query_id,
    col_iteration,
    col_product_id,
    col_ranking_postion,
    col_score,
    col_conf,
]].to_csv(
    os.path.join(output_path, "hypothesis_fine_tuned_triplet_v3.results"),
    index=False,
    header=False,
    sep=' ',
)


""" 2. Generate QRELS file """
df_examples = pd.read_parquet(os.path.join(dataset_path, 'shopping_queries_dataset_examples.parquet'))
df_products = pd.read_parquet(os.path.join(dataset_path, 'shopping_queries_dataset_products.parquet'))
df_examples_products = pd.merge(
    df_examples,
    df_products,
    how='left',
    left_on=[col_product_locale, col_product_id],
    right_on=[col_product_locale, col_product_id]
)
df_examples_products = df_examples_products[df_examples_products[col_small_version] == 1]
df_examples_products = df_examples_products[df_examples_products[col_split] == "test"]

df_examples_products[col_iteration] = 0
df_examples_products[col_relevance_pos] = df_examples_products[col_esci_label].apply(lambda esci_label: esci_label2relevance_pos[esci_label])
df_examples_products = df_examples_products[[
        col_query_id,
        col_iteration,
        col_product_id,
        col_relevance_pos,
]]
df_examples_products.to_csv(
    os.path.join(output_path, "test.qrels"),
    index=False,
    header=False,
    sep=' ',
)



/var/folders/hk/yvd899_50690_61w0jnvdgvw0000gp/T/ipykernel_48204/4015949970.py:48: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for (query_id, rows) in df_results_product_id:


In [11]:
df_examples_products

,query_id,iteration,product_id,relevance_pos
32,2,0,B07D7TBSGH,1
33,2,0,B07DHT2WZK,1
34,2,0,B07DHX8YH2,4
35,2,0,B07DS1YCRZ,2
36,2,0,B07DS3J3MB,2
...,...,...,...,...
2621163,130645,0,B00JET12SW,4
2621164,130645,0,B00JET11BU,4
2621165,130645,0,B007M8TLZI,4
2621166,130645,0,B004ECLQHQ,1


In [12]:
df_res

,query_id,product_id,ranking_postion,score,conf,iteration
0,2,B07R6P8TK8,0,124.632,baseline,Q0
1,2,B07R3TNQDM,1,121.263,baseline,Q0
2,2,B001OJXVKW,2,117.895,baseline,Q0
3,2,B0019Y6TTC,3,114.526,baseline,Q0
4,2,B018JYCD80,4,111.158,baseline,Q0
...,...,...,...,...,...,...
181696,130378,B0054LHI5A,11,32.000,baseline,Q0
181697,130378,B006IB5T4W,12,24.000,baseline,Q0
181698,130378,B07KFLL4X7,13,16.000,baseline,Q0
181699,130378,B01HFH4DAI,14,8.000,baseline,Q0


In [13]:
!ls ./output/

hypothesis.results
hypothesis_baseline.results
hypothesis_baseline_triplet.results
hypothesis_baseline_triplet_cosine.results
hypothesis_baseline_triplet_product_text_cosine.results
hypothesis_fine_tuned_triplet.results
hypothesis_fine_tuned_triplet_v1.results
hypothesis_fine_tuned_triplet_v3.results
test.qrels


In [20]:
# ! less ./output/hypothesis_baseline_triplet.results -n 10

In [ ]:
# ../code/terrier-project-5.5/bin/terrier trec_eval {QRELS_FILE} {RES_FILE} -c -J -m 'ndcg.1=0,2=0.01,3=0.1,4=1'